In [1]:
# Import necessary modules
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
import base64
import os
import numpy as np
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output, State
from animalshelter import AnimalShelter  # Import your CRUD Python module

# Set up MongoDB credentials
username = "aacuser"
password = "SNHU123456"
db = AnimalShelter(username, password)

# Load data from MongoDB into a DataFrame
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

# Create the JupyterDash application
app = JupyterDash(__name__)

# Load Grazioso Salvare's logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define the app layout
app.layout = html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('CS-340 Dashboard - Steven Luu'))),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'wt'},
                {'label': 'Mountain Rescue', 'value': 'mt'},
                {'label': 'Disaster Rescue', 'value': 'ds'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block'}
        )
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="multi",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[],
                         page_action="native",
                         page_current=0,
                         page_size=10),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(id='graph-id',
                          className='col s12 m6'),
                 html.Div(id='map-id',
                          className='col s12 m6',
                          )
             ])
])

# Callbacks for updating data table and graph based on filter type
@app.callback([Output('datatable-id', 'data'),
               Output('datatable-id', 'columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    try:
        print(f"Filter Type: {filter_type}") 
        if filter_type == 'wt':
            df = pd.DataFrame(list(db.read({'$and': [
                {'sex_upon_outcome': 'Intact Male'},
            ]}))) 
        elif filter_type == 'mt':
            df = pd.DataFrame(list(db.read({'$and': [
                {'sex_upon_outcome': 'Intact Male'},
            ]})))
        elif filter_type == 'ds':
            df = pd.DataFrame(list(db.read({'$and': [
                {'sex_upon_outcome': 'Intact Male'}
            ]})))
        elif filter_type == 'reset':
            df = pd.DataFrame.from_records(db.read({}))
        columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data = df.to_dict('records')
        print(f"Filtered Data: {len(df)} rows")   
        return data, columns
    except Exception as e:
        print(f"Error: {str(e)}")
        # Handle the error or log it as needed
        return [], []  # Return empty data and columns in case of an error


# Callback for updating pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return
    dff = pd.DataFrame(viewData)
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']
    fig = px.pie(
        breed_counts,
        names='breed',
        values='count',
        color='breed',
        color_discrete_map={'Australian Shepherd Mix': 'lightcyan',
                            'German Shepherd Mix': 'cyan',
                            'Other': 'royalblue'},
        title='Distribution of Dog Breeds',
        hole=0.3,
    )
    fig.update_traces(textinfo='percent+label', pull=[0.1, 0, 0])
    fig.update_layout(margin=dict(t=0, b=0, l=0, r=0))
    return [
        dcc.Graph(figure=fig)
    ]

# Callback for highlighting selected rows in data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback for updating the geo-location chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or not index:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index and index[0] < len(dff) else 0

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

# Run the app
app.run_server(debug=True, port=30030)


Dash app running on http://127.0.0.1:30030/
Filter Type: reset
Filtered Data: 30000 rows
Filter Type: wt
Filtered Data: 3561 rows
Filter Type: mt
Filtered Data: 3561 rows
Filter Type: ds
Filtered Data: 3561 rows
Filter Type: reset
Filtered Data: 30000 rows
